**This notebook is an exercise in the [Time Series](https://www.kaggle.com/learn/time-series) course.  You can reference the tutorial at [this link](https://www.kaggle.com/ryanholbrook/hybrid-models).**

---


# Introduction #

Run this cell to set everything up!

In [104]:
# Setup feedback system
# from learntools.core import binder
# binder.bind(globals())
# from learntools.time_series.ex5 import *

# Setup notebook
from pathlib import Path
# from learntools.time_series.style import *  # plot style settings

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.deterministic import DeterministicProcess
from xgboost import XGBRegressor


# comp_dir = Path('../input/store-sales-time-series-forecasting')
# data_dir = Path("../input/ts-course-data")

store_sales = pd.read_csv(
    # comp_dir / 
    'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

family_sales = (
    store_sales
    .groupby(['family', 'date'])
    .mean()
    .unstack('family')
    .loc['2017']
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17204\4000327606.py:21: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  store_sales = pd.read_csv(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_17204\4000327606.py:38: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['family', 'date'])


In [105]:
family_sales

sales                                              \
family     AUTOMOTIVE BABY CARE     BEAUTY    BEVERAGES     BOOKS   
date                                                                
2017-01-01   0.092593  0.037037   0.055556    74.222221  0.000000   
2017-01-02  11.481482  0.259259  11.648149  6208.055664  0.481481   
2017-01-03   8.296296  0.296296   7.185185  4507.814941  0.814815   
2017-01-04   6.833333  0.333333   6.888889  3911.833252  0.759259   
2017-01-05   6.333333  0.351852   5.925926  3258.796387  0.407407   
...               ...       ...        ...          ...       ...   
2017-08-11   8.166667  0.129630   6.500000  3502.055664  0.000000   
2017-08-12   7.462963  0.055556   6.833333  3376.259277  0.000000   
2017-08-13   8.907408  0.166667   8.018518  3747.296387  0.018519   
2017-08-14   5.407407  0.166667   6.240741  3237.629639  0.000000   
2017-08-15   6.240741  0.148148   6.277778  3162.462891  0.000000   

                                                                           \
family     BREAD/BAKERY CELEBRATION     CLEANING        DAIRY        DELI   
date                                                                        
2017-01-01     9.084685    0.129630     7.500000    11.518518    3.629167   
2017-01-02   844.836304   14.203704  2233.648193  1545.000000  539.114807   
2017-01-03   665.124146   10.629630  1711.907349  1204.203735  404.300049   
2017-01-04   594.160645   11.185185  1508.036987  1107.796265  309.397675   
2017-01-05   495.511597   12.444445  1241.833374   829.277771  260.776489   
...                 ...         ...          ...          ...         ...   
2017-08-11   522.305298   16.111111  1031.148193   831.648132  345.934967   
2017-08-12   507.473114   15.722222  1072.870361   793.592590  290.553558   
2017-08-13   615.976990   10.074074  1143.648193   928.000000  325.801361   
2017-08-14   518.564026   11.000000  1019.111084   797.222229  271.532257   
2017-08-15   539.966553   11.055555  1082.851807   753.833313  276.582458   

            ... onpromotion                                        \
family      ...   MAGAZINES      MEATS PERSONAL CARE PET SUPPLIES   
date        ...                                                     
2017-01-01  ...         0.0   0.018519      0.111111     0.018519   
2017-01-02  ...         0.0   0.462963     10.592593     0.537037   
2017-01-03  ...         0.0   0.481481      9.722222     0.444444   
2017-01-04  ...         0.0   0.370370     12.037037     0.444444   
2017-01-05  ...         0.0   8.981481      5.666667     0.000000   
...         ...         ...        ...           ...          ...   
2017-08-11  ...         0.0  22.925926      9.000000     0.000000   
2017-08-12  ...         0.0   0.000000      8.703704     0.000000   
2017-08-13  ...         0.0   0.000000      9.240741     0.000000   
2017-08-14  ...         0.0   0.000000      8.722222     0.000000   
2017-08-15  ...         0.0   0.000000      8.814815     0.000000   

                                                                          \
family     PLAYERS AND ELECTRONICS    POULTRY PREPARED FOODS     PRODUCE   
date                                                                       
2017-01-01                0.000000   0.000000       0.037037    0.129630   
2017-01-02                0.000000   0.259259       1.166667    5.629630   
2017-01-03                0.000000   0.388889       1.351852   56.296296   
2017-01-04                0.000000   0.296296       5.444444  101.277778   
2017-01-05                0.000000   0.296296       0.907407    5.018519   
...                            ...        ...            ...         ...   
2017-08-11                0.000000  27.407407       0.537037    6.000000   
2017-08-12                0.018519   0.111111       0.277778    6.000000   
2017-08-13                0.000000   0.185185       0.222222    5.907407   
2017-08-14                0.000000   0.129630       0.148148    6.018519   
2017-08-15     

In [106]:
store_sales

sales  onpromotion
store_nbr family     date                              
1         AUTOMOTIVE 2013-01-01   0.000000            0
                     2013-01-02   2.000000            0
                     2013-01-03   3.000000            0
                     2013-01-04   3.000000            0
                     2013-01-05   5.000000            0
...                                    ...          ...
9         SEAFOOD    2017-08-11  23.830999            0
                     2017-08-12  16.859001            4
                     2017-08-13  20.000000            0
                     2017-08-14  17.000000            0
                     2017-08-15  16.000000            0

[3000888 rows x 2 columns]

In [107]:
family_sales

sales                                              \
family     AUTOMOTIVE BABY CARE     BEAUTY    BEVERAGES     BOOKS   
date                                                                
2017-01-01   0.092593  0.037037   0.055556    74.222221  0.000000   
2017-01-02  11.481482  0.259259  11.648149  6208.055664  0.481481   
2017-01-03   8.296296  0.296296   7.185185  4507.814941  0.814815   
2017-01-04   6.833333  0.333333   6.888889  3911.833252  0.759259   
2017-01-05   6.333333  0.351852   5.925926  3258.796387  0.407407   
...               ...       ...        ...          ...       ...   
2017-08-11   8.166667  0.129630   6.500000  3502.055664  0.000000   
2017-08-12   7.462963  0.055556   6.833333  3376.259277  0.000000   
2017-08-13   8.907408  0.166667   8.018518  3747.296387  0.018519   
2017-08-14   5.407407  0.166667   6.240741  3237.629639  0.000000   
2017-08-15   6.240741  0.148148   6.277778  3162.462891  0.000000   

                                                                           \
family     BREAD/BAKERY CELEBRATION     CLEANING        DAIRY        DELI   
date                                                                        
2017-01-01     9.084685    0.129630     7.500000    11.518518    3.629167   
2017-01-02   844.836304   14.203704  2233.648193  1545.000000  539.114807   
2017-01-03   665.124146   10.629630  1711.907349  1204.203735  404.300049   
2017-01-04   594.160645   11.185185  1508.036987  1107.796265  309.397675   
2017-01-05   495.511597   12.444445  1241.833374   829.277771  260.776489   
...                 ...         ...          ...          ...         ...   
2017-08-11   522.305298   16.111111  1031.148193   831.648132  345.934967   
2017-08-12   507.473114   15.722222  1072.870361   793.592590  290.553558   
2017-08-13   615.976990   10.074074  1143.648193   928.000000  325.801361   
2017-08-14   518.564026   11.000000  1019.111084   797.222229  271.532257   
2017-08-15   539.966553   11.055555  1082.851807   753.833313  276.582458   

            ... onpromotion                                        \
family      ...   MAGAZINES      MEATS PERSONAL CARE PET SUPPLIES   
date        ...                                                     
2017-01-01  ...         0.0   0.018519      0.111111     0.018519   
2017-01-02  ...         0.0   0.462963     10.592593     0.537037   
2017-01-03  ...         0.0   0.481481      9.722222     0.444444   
2017-01-04  ...         0.0   0.370370     12.037037     0.444444   
2017-01-05  ...         0.0   8.981481      5.666667     0.000000   
...         ...         ...        ...           ...          ...   
2017-08-11  ...         0.0  22.925926      9.000000     0.000000   
2017-08-12  ...         0.0   0.000000      8.703704     0.000000   
2017-08-13  ...         0.0   0.000000      9.240741     0.000000   
2017-08-14  ...         0.0   0.000000      8.722222     0.000000   
2017-08-15  ...         0.0   0.000000      8.814815     0.000000   

                                                                          \
family     PLAYERS AND ELECTRONICS    POULTRY PREPARED FOODS     PRODUCE   
date                                                                       
2017-01-01                0.000000   0.000000       0.037037    0.129630   
2017-01-02                0.000000   0.259259       1.166667    5.629630   
2017-01-03                0.000000   0.388889       1.351852   56.296296   
2017-01-04                0.000000   0.296296       5.444444  101.277778   
2017-01-05                0.000000   0.296296       0.907407    5.018519   
...                            ...        ...            ...         ...   
2017-08-11                0.000000  27.407407       0.537037    6.000000   
2017-08-12                0.018519   0.111111       0.277778    6.000000   
2017-08-13                0.000000   0.185185       0.222222    5.907407   
2017-08-14                0.000000   0.129630       0.148148    6.018519   
2017-08-15     

-------------------------------------------------------------------------------

In the next two questions, you'll create a boosted hybrid for the *Store Sales* dataset by implementing a new Python class. Run this cell to create the initial class definition. You'll add `fit` and `predict` methods to give it a scikit-learn like interface.


In [108]:
# You'll add fit and predict methods to this minimal class
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None  # store column names from fit method


# 1) Define fit method for boosted hybrid

Complete the `fit` definition for the `BoostedHybrid` class. Refer back to steps 1 and 2 from the **Hybrid Forecasting with Residuals** section in the tutorial if you need.

In [109]:
def fit(self, X_1, X_2, y):
    # YOUR CODE HERE: fit self.model_1
    self.model_1.fit(X_1, y)

    y_fit = pd.DataFrame(
        # YOUR CODE HERE: make predictions with self.model_1
        self.model_1.predict(X_1),
        index=X_1.index, columns=y.columns,
    )
    
    # YOUR CODE HERE: compute residuals
    y_resid = y - y_fit
    y_resid = y_resid.stack().squeeze() # wide to long

    # YOUR CODE HERE: fit self.model_2 on residuals
    self.model_2.fit(X_2, y_resid)

    # Save column names for predict method
    self.y_columns = y.columns
    # Save data for question checking
    self.y_fit = y_fit
    self.y_resid = y_resid

# Add method to class
BoostedHybrid.fit = fit


-------------------------------------------------------------------------------

# 2) Define predict method for boosted hybrid

Now define the `predict` method for the `BoostedHybrid` class. Refer back to step 3 from the **Hybrid Forecasting with Residuals** section in the tutorial if you need.

In [110]:
def predict(self, X_1, X_2):
    y_pred = pd.DataFrame(
        # YOUR CODE HERE: predict with self.model_1
        self.model_1.predict(X_1),
        index=X_1.index, columns=self.y_columns,
    )
    y_pred = y_pred.stack().squeeze()  # wide to long

    # YOUR CODE HERE: add self.model_2 predictions to y_pred
    y_pred += self.model_2.predict(X_2)
    
    return y_pred.unstack()  # long to wide


# Add method to class
BoostedHybrid.predict = predict



-------------------------------------------------------------------------------

Now you're ready to use your new `BoostedHybrid` class to create a model for the *Store Sales* data. Run the next cell to set up the data for training.

In [111]:
# Target series
y = family_sales.loc[:, 'sales']


# X_1: Features for Linear Regression
dp = DeterministicProcess(index=y.index, order=1)
X_1 = dp.in_sample()


In [112]:
X_1

,trend
date,
2017-01-01,1.0
2017-01-02,2.0
2017-01-03,3.0
2017-01-04,4.0
2017-01-05,5.0
...,...
2017-08-11,223.0
2017-08-12,224.0
2017-08-13,225.0


In [139]:
# X_2: Features for XGBoost
X_2 = family_sales.drop('sales', axis=1).stack()  # onpromotion feature
X_2 = X_2.reset_index('family')
X_2

,family,onpromotion
date,,
2017-01-01,AUTOMOTIVE,0.000000
2017-01-01,BABY CARE,0.000000
2017-01-01,BEAUTY,0.000000
2017-01-01,BEVERAGES,0.703704
2017-01-01,BOOKS,0.000000
...,...,...
2017-08-15,POULTRY,0.111111
2017-08-15,PREPARED FOODS,0.166667
2017-08-15,PRODUCE,58.685185


In [140]:
from sentence_transformers import SentenceTransformer
model_emb = SentenceTransformer('all-MiniLM-L6-v2')

In [141]:
embeddings = model_emb.encode(X_2.family)
embeddings.shape, embeddings

d:\Programs\miniconda3\envs\pt2\lib\site-packages\sentence_transformers\SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]


((7491, 384),
 array([[-0.04261975,  0.03344563,  0.02958248, ...,  0.03304652,
          0.05457871,  0.0502959 ],
        [-0.06718145,  0.01842834, -0.01734119, ..., -0.08300912,
          0.14091533,  0.0151974 ],
        [-0.01402024,  0.06267996,  0.0784174 , ..., -0.04311018,
          0.07030588,  0.00763177],
        ...,
        [-0.03707667,  0.02151926, -0.05466745, ...,  0.01389587,
          0.1902345 ,  0.01624376],
        [-0.06986537,  0.10276181,  0.08544832, ..., -0.00934693,
          0.00572225,  0.00651487],
        [-0.05820651,  0.04147512,  0.00660496, ...,  0.04060396,
          0.07800303,  0.14604555]], dtype=float32))

In [142]:
%%time
X_2['emb'] = [model_emb.encode(row) for row in X_2.family ]

CPU times: total: 47.2 s
Wall time: 47.2 s


In [143]:
X_2

,family,onpromotion,emb
date,,,
2017-01-01,AUTOMOTIVE,0.000000,"[-0.042619765, 0.033445638, 0.029582469, 0.034..."
2017-01-01,BABY CARE,0.000000,"[-0.067181446, 0.018428335, -0.01734117, 0.067..."
2017-01-01,BEAUTY,0.000000,"[-0.014020237, 0.06267997, 0.078417405, 0.0288..."
2017-01-01,BEVERAGES,0.703704,"[-0.008012935, -0.024870131, -0.028842198, 0.0..."
2017-01-01,BOOKS,0.000000,"[-0.02822967, -0.0040181084, -0.012811625, 0.0..."
...,...,...,...
2017-08-15,POULTRY,0.111111,"[0.0020246247, -0.031861294, -0.020025061, 0.0..."
2017-08-15,PREPARED FOODS,0.166667,"[0.014584758, -0.022660904, -0.012866325, 0.14..."
2017-08-15,PRODUCE,58.685185,"[-0.0370767, 0.021519259, -0.05466746, 0.04013..."


In [170]:

X_2df = pd.DataFrame(X_2.emb)
X_2df['new'] = 0
X_2df.loc[0,'new'] = X_2df.iloc[0, 0]


ValueError: Must have equal len keys and value when setting with an iterable

In [158]:
# %%time
X_2e = X_2.explode('emb')
X_2e

TypeError: DataFrame.explode() got an unexpected keyword argument 'axis'

In [151]:
X_2e.stack()
X_2e

,family,onpromotion,emb
date,,,
2017-01-01,AUTOMOTIVE,0.0,-0.04262
2017-01-01,AUTOMOTIVE,0.0,0.033446
2017-01-01,AUTOMOTIVE,0.0,0.029582
2017-01-01,AUTOMOTIVE,0.0,0.03439
2017-01-01,AUTOMOTIVE,0.0,-0.032907
...,...,...,...
2017-08-15,SEAFOOD,0.0,0.080829
2017-08-15,SEAFOOD,0.0,0.036473
2017-08-15,SEAFOOD,0.0,0.040604


In [132]:
df_emb = pd.DataFrame(embeddings)
df_emb

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.042620,0.033446,0.029582,0.034390,-0.032907,0.030346,0.080264,0.069170,-0.022324,-0.026192,...,0.022332,-0.028348,0.030329,-0.052918,-0.033564,-0.046101,0.046696,0.033047,0.054579,0.050296
1,-0.067181,0.018428,-0.017341,0.067066,-0.013721,0.033343,0.047328,-0.025845,-0.026910,0.037479,...,0.038853,0.041575,-0.018708,-0.060958,-0.006371,0.119996,0.047499,-0.083009,0.140915,0.015197
2,-0.014020,0.062680,0.078417,0.028830,-0.031779,-0.012789,0.132203,-0.044997,0.008817,-0.019849,...,0.020423,-0.080530,0.013123,0.069268,0.004674,0.082215,0.162189,-0.043110,0.070306,0.007632
3,-0.008013,-0.024870,-0.028842,0.035618,0.026295,0.033680,0.135792,-0.009846,0.022289,-0.056732,...,0.007625,0.000401,-0.026286,0.002822,-0.059416,-0.066313,0.078938,0.031847,0.091824,-0.018848
4,-0.028230,-0.004018,-0.012812,0.058205,-0.127954,0.029729,0.055368,0.014869,0.051021,0.091689,...,0.031434,0.036262,-0.006173,0.029203,-0.046227,0.013076,0.080745,-0.054872,-0.027286,0.030100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7486,0.002025,-0.031861,-0.020025,0.026727,-0.008250,-0.013260,0.083179,-0.058774,0.062640,-0.010854,...,-0.027888,0.018307,0.037293,-0.023009,0.041382,-0.002058,0.016372,0.065672,0.059623,0.073023
7487,0.014585,-0.022661,-0.012866,0.145712,-0.059989,0.040952,-0.009036,-0.033012,-0.021275,-0.033745,...,0.007193,0.011424,0.047432,-0.000667,0.090169,-0.007176,-0.087634,0.051925,0.030013,0.000067
7488,-0.037077,0.021519,-0.054667,0.040131,-0.001375,0.033182,0.024340,0.020603,-0.018883,-0.017639,...,0.032303,-0.025226,0.014203,-0.050225,-0.005486,0.044358,0.057605,0.013896,0.190234,0.016244
7489,-0.069865,0.102762,0.085448,0.030656,0.009024,0.019003,0.027747,-0.004041,-0.007762,0.017289,...,-0.002145,-0.027495,0.002893,0.002055,0.019733,-0.021569,-0.053248,-0.009347,0.005722,0.006515


In [133]:
# X_2 = X_2.reset_index(drop=True)
X_2

,family,onpromotion
date,,
2017-01-01,AUTOMOTIVE,0.000000
2017-01-01,BABY CARE,0.000000
2017-01-01,BEAUTY,0.000000
2017-01-01,BEVERAGES,0.703704
2017-01-01,BOOKS,0.000000
...,...,...
2017-08-15,POULTRY,0.111111
2017-08-15,PREPARED FOODS,0.166667
2017-08-15,PRODUCE,58.685185


In [138]:
# df_emb = df_emb.reset_index(drop=True)
# X_2 = X_2.reset_index(drop=True)
m = pd.merge(X_2, df_emb, how='left')
# pandas.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

X_2

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [88]:

# Label encoding for 'family'
le = LabelEncoder()  # from sklearn.preprocessing
# X_2 = X_2.reset_index('family')
X_2['family'] = le.fit_transform(X_2['family'])

# Label encoding for seasonality
X_2["day"] = X_2.index.day  # values are day of the month

KeyError: 'family'

In [ ]:
X_2

# 3) Train boosted hybrid

Create the hybrid model by initializing a `BoostedHybrid` class with `LinearRegression()` and `XGBRegressor()` instances.

In [ ]:
# YOUR CODE HERE: Create LinearRegression + XGBRegressor hybrid with BoostedHybrid
model = BoostedHybrid(model_1=LinearRegression(), model_2=XGBRegressor()) # check model name and ()

# YOUR CODE HERE: Fit and predict
model.fit(X_1, X_2, y)
y_pred = model.predict(X_1, X_2)

y_pred = y_pred.clip(0.0)

# Check your answer
q_3.check()

In [ ]:
# Lines below will give you a hint or solution code
q_3.hint()
q_3.solution()

-------------------------------------------------------------------------------

Depending on your problem, you might want to use other hybrid combinations than the linear regression + XGBoost hybrid you've created in the previous questions. Run the next cell to try other algorithms from scikit-learn.

In [ ]:
# Model 1 (trend)
from sklearn.linear_model import ElasticNet, Lasso, Ridge

# Model 2
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Boosted Hybrid
# model = BoostedHybrid(
#     model_1=LinearRegression(), 
#     model_2=XGBRegressor()
# ) 

# Try different combinations of the algorithms above
# model = BoostedHybrid(
#     model_1=Ridge(),
#     model_2=KNeighborsRegressor(),
# )

# Hybrid
model = BoostedHybrid(
    model_1=ElasticNet(), 
    model_2=RandomForestRegressor()
) 

These are just some suggestions. You might discover other algorithms you like in the scikit-learn [User Guide](https://scikit-learn.org/stable/supervised_learning.html).

Use the code in this cell to see the predictions your hybrid makes.

In [ ]:
y_train, y_valid = y[:"2017-07-01"], y["2017-07-02":]
X1_train, X1_valid = X_1[: "2017-07-01"], X_1["2017-07-02" :]
X2_train, X2_valid = X_2.loc[:"2017-07-01"], X_2.loc["2017-07-02":]

# Some of the algorithms above do best with certain kinds of
# preprocessing on the features (like standardization), but this is
# just a demo.
model.fit(X1_train, X2_train, y_train)
y_fit = model.predict(X1_train, X2_train).clip(0.0)
y_pred = model.predict(X1_valid, X2_valid).clip(0.0)

families = y.columns[0:6]
axs = y.loc(axis=1)[families].plot(
    subplots=True, sharex=True, figsize=(11, 9), **plot_params, alpha=0.5,
)
_ = y_fit.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C0', ax=axs)
_ = y_pred.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C3', ax=axs)
for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(family)

# 4) Fit with different learning algorithms

Once you're ready to move on, run the next cell for credit on this question.

In [ ]:
# View the solution (Run this cell to receive credit!)
q_4.check()

# Keep Going #

[**Convert any forecasting task**](https://www.kaggle.com/ryanholbrook/forecasting-with-machine-learning) to a machine learning problem with four ML forecasting strategies.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/time-series/discussion) to chat with other learners.*